In [ ]:
!pip install quantdsl

In [ ]:
%%writefile powerplant.py
from quantdsl.semantics import Choice, TimeDelta, Wait, inline


def PowerPlant(start, end, temp):
    if (start < end):
        Wait(start, Choice(
            PowerPlant(Tomorrow(start), end, Hot()) + ProfitFromRunning(start, temp),
            PowerPlant(Tomorrow(start), end, Stopped(temp))
        ))
    else:
        return 0


@inline
def Power(start):
    DayAhead(start, 'POWER')


@inline
def Gas(start):
    DayAhead(start, 'GAS')


@inline
def DayAhead(start, name):
    ForwardMarket(Tomorrow(start), name)

    
@inline
def Tomorrow(start):
    start + TimeDelta('1d')


@inline
def ProfitFromRunning(start, temp):
    if temp == Cold():
        return 0.3 * Power(start) - Gas(start)
    elif temp == Warm():
        return 0.6 * Power(start) - Gas(start)
    else:
        return Power(start) - Gas(start)


@inline
def Stopped(temp):
    if temp == Hot():
        Warm()
    else:
        Cold()


@inline
def Hot():
    2


@inline
def Warm():
    1


@inline
def Cold():
    0


In [ ]:
gas_and_power = {
    'name': 'quantdsl.priceprocess.blackscholes.BlackScholesPriceProcess',
    'market': ['GAS', 'POWER'],
    'sigma': [0.3, 0.3],
    'rho': [[1.0, 0.8], [0.8, 1.0]],
    'curve': {
            'GAS': [
                ('2012-1-1', 11.0),
                ('2012-1-2', 11.0),
                ('2012-1-3', 1.0),
                ('2012-1-4', 1.0),
                ('2012-1-5', 11.0),
            ],
            'POWER': [
                ('2012-1-1', 1.0),
                ('2012-1-2', 1.0),
                ('2012-1-3', 11.0),
                ('2012-1-4', 11.0),
                ('2012-1-5', 10.0),
            ]
        }
}

In [ ]:
from quantdsl.interfaces.calcandplot import calc_print

results = calc_print(
    source_code=("from powerplant import PowerPlant, Cold, Date\n"
                 "PowerPlant(Date('2012-1-1'), Date('2012-1-5'), Cold())"),
    observation_date='2011-1-1',
    interest_rate=2.5,
    periodisation='daily',
    price_process=gas_and_power,
    verbose=True
)